# Bayesian Classification By Phoneme Frequency
Camille Girard

Jonah Spicher

Question: Let's asy you are in a place where people are speaking five languages: English, Spanish, Italian, Mandarin, Arabic, and Hindi. You overhear someone say a word, based on the sounds in that word, which language are they speaking? 
Given each of these five words, (language, idioma, linguaggio, 语言, لغة, भाषा)